# ARC TTT (Test-Time Training)

Ensemble solver with 3 solvers x 2 runs each (6 candidates total):
- **Perceiver**: Perception-based solver
- **Phased**: Structured tool-calling solver  
- **Iterative**: Code execution with iterative refinement

Voting + Judge selects top 2 submissions.

In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

import json
import sys
import glob
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

import numpy as np

KAGGLE = Path("/kaggle/input").exists()

if KAGGLE:
    ENSEMBLE_DIR = "/kaggle/input/arc-ttt/arc_ttt"
    EVAL_DIR = "/kaggle/input/arc-prize-2025/arc-agi_evaluation_challenges"
    OUTPUT_DIR = "/kaggle/working"
    
    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        os.environ["OPENROUTER_API_KEY"] = user_secrets.get_secret("OPENROUTER_API_KEY")
        print("✓ API key loaded from Kaggle secrets")
    except Exception as e:
        print(f"⚠️ Failed to load API key from secrets: {e}")
        os.environ["OPENROUTER_API_KEY"] = ""
else:
    # Local development environment
    ENSEMBLE_DIR = "/Users/sarooshkhan/Desktop/learning/arc_ttt_claude"
    EVAL_DIR = "/Users/sarooshkhan/Desktop/learning/past runs/arc main/data/evaluation"
    OUTPUT_DIR = ENSEMBLE_DIR
    
    # Load from environment or use stored key
    os.environ["OPENROUTER_API_KEY"] = os.environ.get("OPENROUTER_API_KEY", "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")

if not os.environ.get("OPENROUTER_API_KEY"):
    raise ValueError("OPENROUTER_API_KEY not found. Set it as an environment variable or use Kaggle secrets.")

sys.path.insert(0, ENSEMBLE_DIR)
os.chdir(ENSEMBLE_DIR)

from orchestrator import solve_with_judge, solve_batch_with_judge
from config import reset_usage_stats, get_usage_stats

print("=" * 60)
print("🚀 ARC TTT SOLVER")
print("=" * 60)
print(f"Environment: {'Kaggle' if KAGGLE else 'Local'}")
print(f"Ensemble Dir: {ENSEMBLE_DIR}")
print(f"Eval Dir: {EVAL_DIR}")
print(f"Output Dir: {OUTPUT_DIR}")

🚀 ARC TTT SOLVER
Environment: Local
Ensemble Dir: /Users/sarooshkhan/Desktop/learning/arc_ttt_claude
Eval Dir: /Users/sarooshkhan/Desktop/learning/past runs/arc main/data/evaluation
Output Dir: /Users/sarooshkhan/Desktop/learning/arc_ttt_claude


In [2]:
MAX_CONCURRENT = 50

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
RESULTS_FILE = os.path.join(OUTPUT_DIR, f"results_{timestamp}.jsonl")
SUBMISSION_FILE = os.path.join(OUTPUT_DIR, f"submission_{timestamp}.json")

print(f"Results file: {RESULTS_FILE}")
print(f"Submission file: {SUBMISSION_FILE}")

submission_data = {}
FALLBACK = [[0, 0], [0, 0]]

Results file: /Users/sarooshkhan/Desktop/learning/arc_ttt_claude/results_20260205_185314.jsonl
Submission file: /Users/sarooshkhan/Desktop/learning/arc_ttt_claude/submission_20260205_185314.json


In [3]:
def load_task(task_id: str) -> dict:
    task_path = os.path.join(EVAL_DIR, f"{task_id}.json")
    with open(task_path) as f:
        return json.load(f)

def get_ground_truth_from_task(task_data: dict) -> list[np.ndarray]:
    outputs = []
    for test in task_data.get('test', []):
        if 'output' in test:
            outputs.append(np.array(test['output']))
    return outputs if outputs else None

all_task_files = glob.glob(os.path.join(EVAL_DIR, "*.json"))
all_task_ids = [os.path.basename(f).replace(".json", "") for f in all_task_files]

print(f"Found {len(all_task_ids)} tasks in evaluation directory")

Found 120 tasks in evaluation directory


In [4]:
# Run all tasks
TASKS = all_task_ids

print(f"🎯 Running on ALL {len(TASKS)} tasks")

🎯 Running on ALL 120 tasks


In [5]:
def check_attempt(attempt, ground_truths: list[np.ndarray]) -> tuple[bool, int, int]:
    if attempt is None or ground_truths is None or len(ground_truths) == 0:
        return (False, 0, len(ground_truths) if ground_truths else 0)
    
    n_tests = len(ground_truths)
    
    try:
        if n_tests == 1:
            attempt_arr = np.array(attempt)
            gt = ground_truths[0]
            correct = attempt_arr.shape == gt.shape and np.array_equal(attempt_arr, gt)
            return (correct, 1 if correct else 0, 1)
        else:
            is_list_of_grids = (
                isinstance(attempt, list) and 
                len(attempt) > 0 and 
                isinstance(attempt[0], list) and 
                len(attempt[0]) > 0 and
                isinstance(attempt[0][0], list)
            )
            
            if not is_list_of_grids:
                attempt_arr = np.array(attempt)
                gt = ground_truths[0]
                correct = attempt_arr.shape == gt.shape and np.array_equal(attempt_arr, gt)
                return (False, 1 if correct else 0, n_tests)
            else:
                tests_correct = 0
                for i, gt in enumerate(ground_truths):
                    if i < len(attempt):
                        attempt_arr = np.array(attempt[i])
                        gt_arr = np.array(gt)
                        if attempt_arr.shape == gt_arr.shape and np.array_equal(attempt_arr, gt_arr):
                            tests_correct += 1
                return (tests_correct == n_tests, tests_correct, n_tests)
    except Exception:
        return (False, 0, n_tests)


def calculate_task_score(result: dict, ground_truths: list[np.ndarray]) -> tuple[float, float, float]:
    if ground_truths is None or len(ground_truths) == 0:
        return (0.0, 0.0, 0.0)
    
    attempt_1 = result.get("attempt_1")
    attempt_2 = result.get("attempt_2")
    
    _, tests_correct_1, total_tests = check_attempt(attempt_1, ground_truths)
    _, tests_correct_2, _ = check_attempt(attempt_2, ground_truths)
    
    score_1 = tests_correct_1 / total_tests if total_tests > 0 else 0.0
    score_2 = tests_correct_2 / total_tests if total_tests > 0 else 0.0
    final_score = max(score_1, score_2)
    
    return (score_1, score_2, final_score)

## Run Tasks

In [ ]:
import gc

reset_usage_stats()
results = []
total_score = 0
score_1_total = 0
score_2_total = 0
partial_count = 0  
completed = 0

source_counts = {"attempt_1": {}, "attempt_2": {}}
solver_wins = {"perceiver_1": 0, "perceiver_2": 0, "phased_1": 0, "phased_2": 0, "iterative_1": 0, "iterative_2": 0}

def update_submission_file(task_id, result, task_data):
    global submission_data
    
    n_tests = len(task_data.get('test', []))
    attempt_1 = result.get("attempt_1")
    attempt_2 = result.get("attempt_2")
    
    preds = []
    for i in range(n_tests):
        if n_tests == 1:
            a1 = attempt_1 if attempt_1 is not None else FALLBACK
            a2 = attempt_2 if attempt_2 is not None else FALLBACK
        else:
            if isinstance(attempt_1, list) and len(attempt_1) > i and isinstance(attempt_1[i], list):
                a1 = attempt_1[i]
            else:
                a1 = FALLBACK
            if isinstance(attempt_2, list) and len(attempt_2) > i and isinstance(attempt_2[i], list):
                a2 = attempt_2[i]
            else:
                a2 = FALLBACK
        preds.append({"attempt_1": a1, "attempt_2": a2})
    
    submission_data[task_id] = preds
    
    with open(SUBMISSION_FILE, 'w') as f:
        json.dump(submission_data, f)
    
    return len(submission_data)

task_cache = {}
gt_cache = {}

def process_task(task_id):
    task_data = load_task(task_id)
    gt = get_ground_truth_from_task(task_data)
    task_cache[task_id] = task_data
    gt_cache[task_id] = gt
    return solve_with_judge(task_data, task_id, gt, verbose=True)

with ThreadPoolExecutor(max_workers=MAX_CONCURRENT) as executor:
    futures = {executor.submit(process_task, tid): tid for tid in TASKS}
    
    for future in as_completed(futures):
        task_id = futures[future]
        try:
            result = future.result()
            result["task_id"] = task_id
            results.append(result)
            
            # Calculate score using cached ground truth
            gt = gt_cache.get(task_id) or get_ground_truth_from_task(load_task(task_id))
            score_1, score_2, task_score = calculate_task_score(result, gt)
            total_score += task_score
            score_1_total += score_1
            score_2_total += score_2
            
            # Track sources
            a1_src = result.get("attempt_1_source", "none")
            a2_src = result.get("attempt_2_source", "none")
            source_counts["attempt_1"][a1_src] = source_counts["attempt_1"].get(a1_src, 0) + 1
            source_counts["attempt_2"][a2_src] = source_counts["attempt_2"].get(a2_src, 0) + 1
            
            # Track which solver won
            if score_1 == 1.0:
                solver_wins[a1_src] = solver_wins.get(a1_src, 0) + 1
            elif score_2 == 1.0:
                solver_wins[a2_src] = solver_wins.get(a2_src, 0) + 1
            
            completed += 1
            
            scores_detail = result.get("scores", {})
            a1_scores = scores_detail.get("attempt_1", {})
            a2_scores = scores_detail.get("attempt_2", {})
            a1_partial = f" ({a1_scores.get('tests_correct', 0)}/{a1_scores.get('total_tests', 1)})" if a1_scores.get("partial") else ""
            a2_partial = f" ({a2_scores.get('tests_correct', 0)}/{a2_scores.get('total_tests', 1)})" if a2_scores.get("partial") else ""
            
            if 0 < task_score < 1.0:
                partial_count += 1
            
            if task_score == 1.0:
                status = "✓ PASS"
            elif task_score > 0:
                status = "⚠ PARTIAL"
            else:
                status = "✗ FAIL"
            print(f"[{completed}/{len(TASKS)}] {task_id}: {status} (s1={score_1:.2f} s2={score_2:.2f} best={task_score:.2f}) | a1={a1_src} a2={a2_src}")
            
            with open(RESULTS_FILE, "a") as f:
                f.write(json.dumps({
                    "task_id": task_id,
                    "attempt_1": result.get("attempt_1"),
                    "attempt_2": result.get("attempt_2"),
                    "attempt_1_source": a1_src,
                    "attempt_2_source": a2_src,
                    "ratings": [(r[0], r[1], r[2][:100] if len(r) > 2 and r[2] else "") for r in result.get("ratings", [])],
                    "score_1": score_1,
                    "score_2": score_2,
                    "final_score": task_score,
                    "scores": result.get("scores", {}),  # Detailed scores with tests_correct, total_tests, partial
                    "num_distinct": result.get("num_distinct", 0),
                    "total_candidates": result.get("total_candidates", 6),
                    "distinct_outputs": result.get("distinct_outputs", []),
                    "judge_used": result.get("judge_used", False),
                    "timing": result.get("timing"),
                    "usage": result.get("usage"),
                }) + "\n")
            
            # Update submission file (auto-updates as each task completes)
            task_data = task_cache.get(task_id) or load_task(task_id)
            n_submitted = update_submission_file(task_id, result, task_data)
            print(f"    💾 Submission updated: {n_submitted} tasks")
            
            # Flush memory after each task
            if 'all_outputs' in result:
                del result['all_outputs']  # Large object no longer needed
            if task_id in task_cache:
                del task_cache[task_id]  # Clear cached task data
            gc.collect()
            
        except Exception as e:
            import traceback
            print(f"[{completed}/{len(TASKS)}] {task_id}: ERROR - {e}")
            traceback.print_exc()
            completed += 1

print(f"\n{'='*60}")
print(f"FINAL RESULTS")
print(f"{'='*60}")
print(f"  Score 1 (Attempt 1): {score_1_total:.2f}/{len(TASKS)} = {score_1_total/len(TASKS)*100:.1f}%")
print(f"  Score 2 (Attempt 2): {score_2_total:.2f}/{len(TASKS)} = {score_2_total/len(TASKS)*100:.1f}%")
print(f"  Combined Score:      {total_score:.2f}/{len(TASKS)} = {total_score/len(TASKS)*100:.1f}%")
print(f"  Partial Tasks:       {partial_count}/{len(TASKS)} (tasks with 0 < score < 1)")
print(f"\n  Solver Win Counts:")
for solver, wins in sorted(solver_wins.items(), key=lambda x: -x[1]):
    print(f"    {solver}: {wins}")
print(f"\n  Attempt 1 Sources:")
for src, cnt in sorted(source_counts["attempt_1"].items(), key=lambda x: -x[1]):
    print(f"    {src}: {cnt}")
print(f"\n  Attempt 2 Sources:")
for src, cnt in sorted(source_counts["attempt_2"].items(), key=lambda x: -x[1]):
    print(f"    {src}: {cnt}")
print(f"\n{'='*60}")
print(get_usage_stats())

[da515329] Starting... [e12f9a14] Starting...
TRIPLE SOLVER x2 + JUDGE: Task da515329

🚀 Running all solvers in parallel...
    Iterative×2 start immediately
    Perception runs once, shared by Perceiver×2 + Phased×2
 [a251c730] Starting... 
TRIPLE SOLVER x2 + JUDGE: Task e12f9a14

🚀 Running all solvers in parallel...
    Iterative×2 start immediately
    Perception runs once, shared by Perceiver×2 + Phased×2
[88e364bc] Starting...   👁️ Perception: Starting shared pipeline...
[9385bd28] Starting...   👁️ Perception: Starting shared pipeline...

TRIPLE SOLVER x2 + JUDGE: Task a251c730

🚀 Running all solvers in parallel...
    Iterative×2 start immediately
    Perception runs once, shared by Perceiver×2 + Phased×2
[409aa875] Starting... 
TRIPLE SOLVER x2 + JUDGE: Task 88e364bc

🚀 Running all solvers in parallel...
    Iterative×2 start immediately
    Perception runs once, shared by Perceiver×2 + Phased×2
[abc82100] Starting...   🔵 Iterative: Starting (iterative refinement solver)...
[a6f

Traceback (most recent call last):
  File "/Users/sarooshkhan/Desktop/learning/arc_ttt_claude/solvers/perceiver/solve/tool_solver.py", line 713, in solve_task_with_tools
    test_input = np.array(test_case['input'])
                          ~~~~~~~~~^^^^^^^^^
IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices


Tool called: verify_hypothesis

✓ PHASE 3 COMPLETE: Verification finished

Results:
  ✓ Example 1: PASS
  ✓ Example 2: PASS
  ✓ Example 3: PASS

All examples pass: True
Confidence: HIGH

✓ Ready to implement!
→ Now call implement_solution with code AND predicted output grids

--- Phase 4: Expecting implement_solution ---

Model reasoning:


The analysis confirms my hypothesis perfectly. Let me now implement the transformation:
- **Key sh...

→ Tool call: apply_transformation
  Rule: Extract the 5-bordered box. For each solid block of color C inside the box: (1) find C's template pa...
  Example: 0
← Result: correct

Iteration 3/25

Model reasoning:




→ Tool call: run_code
  Purpose: Find both rectangles in each example (including ones whose border color appears in background tile),...
  Code: 
import numpy as np
from scipy.ndimage import label as ndimage_label

for i, ex in enumerate(train_e...
← Result: success
  Output: ============================================================
E

Traceback (most recent call last):
  File "/Users/sarooshkhan/Desktop/learning/arc_ttt_claude/solvers/perceiver/solve/tool_solver.py", line 713, in solve_task_with_tools
    test_input = np.array(test_case['input'])
                          ~~~~~~~~~^^^^^^^^^
IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices


     [claude-opus-4.6] Try 1/5: ✓ Passed training!
     [claude-opus-4.6] ⚡ Early stop - got first passing solution
     [claude-opus-4.6] 📋 1 solution(s) passed training
     [claude-opus-4.6] ⏭️ Skipping self-verification
  ✓ Perceiver: Solution found (score: 75)

✓ PHASE 1 COMPLETE: Observations recorded

Key patterns identified:
  • Each input has two components: a small sparse pattern (individual colored pixels on background, spaced 2 apart in a grid) and large uniform NxN solid-color blocks elsewhere
  • The small pattern acts as a blueprint/template that defines a grid of colors
  • The large blocks in the input represent ONE row of the blueprint, scaled up to NxN blocks with gaps matching the block spacing
  • The output replicates ALL rows of the blueprint as large blocks, preserving the existing blocks and adding new ones
  • Block size is determined from the existing large blocks (3x3 in Ex0, 2x2 in Ex1/Ex2)
  • The gap between blocks matches the background gap between them 

...

Traceback (most recent call last):
  File "/Users/sarooshkhan/Desktop/learning/arc_ttt_claude/solvers/perceiver/solve/tool_solver.py", line 713, in solve_task_with_tools
    test_input = np.array(test_case['input'])
                          ~~~~~~~~~^^^^^^^^^
IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices


Tool called: hypothesize_rule

✓ PHASE 2 COMPLETE: Hypotheses formulated

→ Hypothesis #1: Each rectangle has two parallel 9-rails. The rectangle slides toward the far rai...
  Hypothesis #2: Each rectangle slides toward its far 9-rail by D cells where D = the gap between...
  Hypothesis #3: Each rectangle moves toward the far rail such that the near rail ends up 1 cell ...

Chosen: Hypothesis #1
Reasoning: Hypothesis 1 best explains the movement distance calculation. The rectangle moves until it's adjacent to the far rail. Let me verify: Ex1 left panel: the 9-block spans rows 1-2. The frame starts at row 3. So the near rail is row 2, far rail is row 1. Distance from frame top (row 3) to far rail (row 1) = 2, minus 1 = 1. Movement = 1 up. Correct. Ex2 top: near rail col 9, far rail col 4. Distance from frame left edge (col 10) to far rail (col 4) = 6, minus 1 = 5. But frame moved from cols 10-14 to cols 7-11, that's only 3. Hmm, that doesn't work either. Let me reconsider. Actually in 

Traceback (most recent call last):
  File "/Users/sarooshkhan/Desktop/learning/arc_ttt_claude/solvers/perceiver/solve/tool_solver.py", line 713, in solve_task_with_tools
    test_input = np.array(test_case['input'])
                          ~~~~~~~~~^^^^^^^^^
IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices



Model reasoning:
Examples 0 and 2 match perfectly! Example 1 (n=3) fails. The formula works for n=1,2 but not n=3. Le...

→ Tool call: run_code
  Purpose: Analyze shell patterns for all examples to find the correct general formula
  Code: 
import numpy as np

def get_shell_values(out, r0, c0, d):
    """Get values and positions going clo...
← Result: success
  Output: 
Example 0: center=(4,5), n=1, grid=12x...

Iteration 4/25
     ✓ Perceiver generated hypotheses

📝 Phase 1 Complete - Analysis ready for Phase 2

PHASE 2: Testing & Refinement with Tools

PHASE 0: Structured Observation (MANDATORY)
  📝 Forcing observe_examples call...
     ✓ Perceiver generated hypotheses

📝 Phase 1 Complete - Analysis ready for Phase 2

PHASE 2: Testing & Refinement with Tools

PHASE 0: Structured Observation (MANDATORY)
  📝 Forcing observe_examples call...
     [claude-opus-4.6] Try 2/5: ✗ No code
Tool called: implement_solution

Explanation: Each non-background object is pushed vertically to either t

Traceback (most recent call last):
  File "/Users/sarooshkhan/Desktop/learning/arc_ttt_claude/solvers/perceiver/solve/tool_solver.py", line 713, in solve_task_with_tools
    test_input = np.array(test_case['input'])
                          ~~~~~~~~~^^^^^^^^^
IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Traceback (most recent call last):
  File "<string>", line 264, in <module>
  File "<string>", line 230, in transform
IndexError: list index out of range
Traceback (most recent call last):
  File "/Users/sarooshkhan/Desktop/learning/arc_ttt_claude/solvers/perceiver/solve/tool_solver.py", line 713, in solve_task_with_tools
    test_input = np.array(test_case['input'])
                          ~~~~~~~~~^^^^^^^^^
IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Traceback (most recent call last):
  File "/Users/sarooshkhan/De